# Automatic Centerlines Extraction


First, extract the coronary artery centerlines from your dataset using the method described in 
[Coronary artery centerline extraction in cardiac CT angiography using a CNN-based orientation classifier (Wolternik et al., 2019b)](https://www.sciencedirect.com/science/article/pii/S1361841518308491). For now, CCTA scans from `<dataset_dir>` are matched with the patterns `*.mhd`
and `*_image.nii.gz`. 

```bash
source scripts/centerlines_autoextract.sh <dataset_dir> <output_dir>
```

For every `datapoint_X` in `<dataset_dir>`, this command creates files `<output_dir>/<datapoint_X>/vessel0.txt`,
`<output_dir>/<datapoint_X>/vessel1.txt` etc., where each `vessel<N>.txt` file corresponds to a discovered coranary 
artery vessel and it contains an array of 4D points (3D XYZ coordinates + artery lumen radius estimate).

## Create the HD5-encoded Dataset

The HD5 file type allows on-the-fly loading of big datasets, without the need to load entire arrays in memory -- 
relevant parts are loaded as needed, reducing memory footprint. 

For more inforamation, refer to the 
[HD5 Python documentation](https://docs.h5py.org/en/stable/).

In [ ]:
%cd ../

/home/marco/contrast-gan-3D


Adjust the variables in the following cell to match your setup:

In [ ]:
from pathlib import Path

DATASET_DIR = Path("/home/marco/data/ASOCA_Philips/images")

OUTPUT_DIR = DATASET_DIR / "auto_centerlines"

# By default, .h5 converted files are saved inside `DATASET_DIR`; set this
# variable if you want to save them somewhere else
H5_OUTPUT_DIR = None

print(str(DATASET_DIR))
print(str(OUTPUT_DIR))
print(str(H5_OUTPUT_DIR))

/home/marco/data/ASOCA_Philips/images
/home/marco/data/ASOCA_Philips/images/auto_centerlines
None


In [ ]:
from pprint import pprint

cctas = sorted(
    list(DATASET_DIR.glob("*.mhd")) + list(DATASET_DIR.glob("*_image.nii.gz"))
)
centerlines = sorted([d for d in OUTPUT_DIR.glob("*") if d.is_dir()])
print(f"Found {len(cctas)} CCTAs with {len(centerlines)} centerlines")

zipped = list(zip(cctas, centerlines))
pprint(zipped[:3])

Found 60 CCTAs with 60 centerlines
[(PosixPath('/home/marco/data/ASOCA_Philips/images/ASOCA-000.mhd'),
  PosixPath('/home/marco/data/ASOCA_Philips/images/auto_centerlines/ASOCA-000')),
 (PosixPath('/home/marco/data/ASOCA_Philips/images/ASOCA-001.mhd'),
  PosixPath('/home/marco/data/ASOCA_Philips/images/auto_centerlines/ASOCA-001')),
 (PosixPath('/home/marco/data/ASOCA_Philips/images/ASOCA-002.mhd'),
  PosixPath('/home/marco/data/ASOCA_Philips/images/auto_centerlines/ASOCA-002'))]


In [ ]:
from contrast_gan_3D.utils import io_utils, logging_utils

logging_utils.set_project_loggers_level(level="DEBUG")

contrast_gan_3D.utils.geometry: INFO -> DEBUG
contrast_gan_3D.utils.io_utils: INFO -> DEBUG


In [ ]:
for ccta, centerlines_dir in zipped:
    if io_utils.stem(centerlines_dir) == io_utils.stem(ccta):
        io_utils.sitk_to_h5(
            ccta,
            centerlines_dir,
            centerlines_dir / "ostia.xml",
            h5_output_dir=H5_OUTPUT_DIR,
        )
        print()
    else:
        print(f"**No centerlines for {str(ccta)!r}**")

[2024-02-02 15:53:27,824: DEBUG] Original image dtype uint16 range (0, 2683) (contrast_gan_3D.utils.io_utils:89)
[2024-02-02 15:53:28,002: DEBUG] New image dtype int16 range (-1024, 1659) (contrast_gan_3D.utils.io_utils:96)
[2024-02-02 15:53:28,049: DEBUG] CCTA: (512, 512, 204) centerlines: (15239, 4) ostia: (2, 3) (contrast_gan_3D.utils.io_utils:130)
[2024-02-02 15:53:28,051: DEBUG] H5 file: '/home/roel/data/ASOCA_Philips/images/ASOCA-000.h5' (contrast_gan_3D.utils.io_utils:139)

